# SWP Index Web Scraping

Jupyter Notebook for the creation of the SWP Index with Web Scraping.

SWP means Size of the Wikipedia Page, and it's an estimation of the importance of a city based on the lenght of its Wikipedia page.

____________________________________________________________

For this process, the following CSV files is needed: ```out_COCI_citations_and_locations_analysis_ready.csv```.

The CSV file must be generated running the ```Citation Datasets Separation for the Analysis.ipynb``` Notebook that is contained in the ```8 - Citation Datasets Separation for the Analysis``` folder of this Repository.

____________________________________________________________

In particular, the following operations are going to be executed:
* Opening of the CSV dataset
* Extraction of the distinct cities
* Web scraping of the wikipedia page's size

Lastly, the entire processed dump is going to be saved on disk in CSV format

In [12]:
# Libraries Import
import pandas as pd
import numpy as np
import platform
import multiprocessing as mp 
import concurrent       
from turistic_scraper_multithread_utils import * 

pd.set_option('display.max_columns', None)

## File Paths
Please set your working directory paths.

In [2]:
# ******************* PATHS ********************+

# Dumps Directory Path
path_file_import = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Import/'

# CSV Exports Directory Path
path_file_export = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/'

### Multithreading Settings
Settings needed for the multithreaded queries to gather the size of the wikipedia pages.

Please specify the max number of workers below:

We suggest to **set the number of workers to 1 if you have a large bandwidth** (over 100Mbps) to limit the requests to the website. Otherwise, you could try to set a higher value to make requests in parallel.

In [3]:
MAX_WORKERS = 2

You can also set a **sleep delay** (in seconds) between requests if that's not enough:

In [4]:
SLEEP_DELAY = 0.2 # Seconds

Special setting for the specific operating systems.

**Note**: Due to the latest MacOS releases' security measures, we need to use the spawn method instead of fork.

In [5]:
print(f"Notebook running on {platform.system()} OS: ")

if platform.system() == "Darwin" or platform.system() == "Windows": # MacOS and windows
    mp_ctx = mp.get_context("spawn")
    print("Spawn method has been set")
    
else: # other unix systems
    mp_ctx = mp.get_context("fork")
    print("Fork method has been set")

Notebook running on Darwin OS: 
Spawn method has been set


## Read of the CSV Dataset

In [6]:
df_coci_dataset = pd.read_csv(path_file_import + 'out_COCI_citations_and_locations_analysis_ready.csv', low_memory=False, index_col=[0])
df_coci_dataset

,CitationCount,ConferenceLocation,ConferenceNormalizedName,ConferenceSeriesNormalizedName,Doi,Year
0,10,"Austin, Texas, United States",disc 2014,disc,10.1007/978-3-662-45174-8_28,2014
1,5,"Wrocław, Lower Silesian Voivodeship, Poland",esa 2014,esa,10.1007/978-3-662-44777-2_60,2014
2,11,"Innsbruck, Tyrol, Austria",enter 2013,enter,10.1007/978-3-319-03973-2_13,2013
3,1,"Villefranche-sur-Saône, Auvergne-Rhône-Alpes, ...",dexa 2002,dexa,10.1007/3-540-46146-9_77,2002
4,9,"Zakopane, Lesser Poland Voivodeship, Poland",icaisc 2006,icaisc,10.1007/11785231_94,2006
...,...,...,...,...,...,...
3107878,4,"Thessaloniki, Macedonia and Thrace, Greece",sapere 2011,sapere,10.1007/978-3-642-31674-6_9,2011
3107879,4,"Thessaloniki, Macedonia and Thrace, Greece",sapere 2011,sapere,10.1007/978-3-642-31674-6_20,2011
3107880,2,"Thessaloniki, Macedonia and Thrace, Greece",sapere 2011,sapere,10.1007/978-3-642-31674-6_25,2011
3107881,0,"Thessaloniki, Macedonia and Thrace, Greece",sapere 2011,sapere,10.1007/978-3-642-31674-6_12,2011


## Obtaining the Size of the Wikipedia Page
The conferences locations are going to be obtained from queries to the Wikipedia Website.

First we need to obtain the distinct ciry names:

In [7]:
df_cities = df_coci_dataset['ConferenceLocation'].copy().to_frame()
df_cities = df_cities.drop_duplicates()

df_cities["City"] = np.nan

df_cities.City = df_cities.ConferenceLocation.str.split(',').str[0].str.split(' ').str[0]
df_cities = df_cities.drop_duplicates()

df_cities

,ConferenceLocation,City
0,"Austin, Texas, United States",Austin
1,"Wrocław, Lower Silesian Voivodeship, Poland",Wrocław
2,"Innsbruck, Tyrol, Austria",Innsbruck
3,"Villefranche-sur-Saône, Auvergne-Rhône-Alpes, ...",Villefranche-sur-Saône
4,"Zakopane, Lesser Poland Voivodeship, Poland",Zakopane
...,...,...
3103379,"Veneto, Italy",Veneto
3103468,"Bastia, Corsica, France",Bastia
3103514,"Laramie, Wyoming, United States",Laramie
3103806,"Longyearbyen, Norway",Longyearbyen


### Define of the Web Scraping Function
The web function is going to be passed as parameter.

In [13]:
def swp_scraper(cities_dataframe, mt_downloader_operation_function, wikipedia_url = "https://en.wikipedia.org/wiki/"):
    dict_city_swp = {}      
    download_list = zip(list(cities_dataframe.ConferenceLocation.values), list(cities_dataframe.City.values))

    executor = concurrent.futures.ProcessPoolExecutor(max_workers=int(MAX_WORKERS), mp_context=mp_ctx)
    futures = [executor.submit(mt_downloader_operation_function, conf_location, wikipedia_url, SLEEP_DELAY) for conf_location in download_list]

    for future in concurrent.futures.as_completed(futures):
        try:
            k, v = future.result()
        except Exception as e:
            print(f"{futures[future]} throws {e}")
        else:
            dict_city_swp[k] = v

    # Converting the resulting dictionary to a dataframe
    df_city_swp = pd.DataFrame(dict_city_swp.items(), columns=['Location', 'SWP'])

    return df_city_swp

#### Queries to the DBLP Website

Parallel execution of the queries to the Wikipedia website.

**Note**: this operation should take less than a 5 minutes, depending on your Internet speed.

In [14]:
df_city_swp = swp_scraper(df_cities, mt_get_city_swp_operation_v1, "https://en.wikipedia.org/wiki/")

In [15]:
df_city_swp

,Location,SWP
0,"(Austin, Texas, United States, Austin)",178769
1,"(Wrocław, Lower Silesian Voivodeship, Poland, ...",136404
2,"(Innsbruck, Tyrol, Austria, Innsbruck)",73456
3,"(Villefranche-sur-Saône, Auvergne-Rhône-Alpes,...",27815
4,"(Zakopane, Lesser Poland Voivodeship, Poland, ...",43925
...,...,...
2465,"(Veneto, Italy, Veneto)",93437
2466,"(Bastia, Corsica, France, Bastia)",64516
2467,"(Laramie, Wyoming, United States, Laramie)",10391
2468,"(Longyearbyen, Norway, Longyearbyen)",69379


In [18]:
df_city_swp["ConferenceLocation"] = np.nan
df_city_swp["City"] = np.nan

df_city_swp.ConferenceLocation = df_city_swp.Location.tuple[0]
df_city_swp.City = df_city_swp.Location.tuple[1]

df_city_swp

AttributeError: 'Series' object has no attribute 'tuple'

Let's see how many conference locations have been fixed.

In [ ]:
df_conf_locations = df_conf_locations.dropna(subset = ['ConferenceLocation'])

print(f"Fixed {len(df_conf_locations.index)} over {len(df_dblp_conferences.index)} unique conferences")

## Join of the New Location Data with the Original Dataframe

In [ ]:
# Merge with the location dataframe
df_dblp_preprocessed = pd.merge(df_dblp_preprocessed, df_conf_locations, on=['url'], how='left')

# Combine the two columns
df_dblp_preprocessed['ConferenceLocation_x'] = df_dblp_preprocessed['ConferenceLocation_x'].fillna(df_dblp_preprocessed['ConferenceLocation_y'])
df_dblp_preprocessed.rename(columns = {'ConferenceLocation_x':'ConferenceLocation'}, inplace=True)
df_dblp_preprocessed = df_dblp_preprocessed.drop(columns=['ConferenceLocation_y'])

df_dblp_preprocessed.iloc[:5]

Count of how many paper's conference locations are still missing

In [ ]:
n_missing = len(df_dblp_preprocessed.index) - len(df_dblp_preprocessed.dropna(subset = ['ConferenceLocation']).index)
print(f"{n_missing} missing paper's conference locations")

## Cleaning the Conference Locations
Some conferences are divided in separated volumes. The volume number is usually indicated in the header of the DBLP page, following the conference location.

For example: *OTM 2004: Agia Napa, Cyprus - Volume 2*

We need to filter it.

In [ ]:
df_dblp_preprocessed.ConferenceLocation = df_dblp_preprocessed.ConferenceLocation.str.split(' - ').str[0]

df_dblp_preprocessed.iloc[:5]

## Write of the Final CSV on Disk

Saving the resulting dataframe on disk in CSV format.

In [ ]:
# Write of the resulting CSV on Disk
df_dblp_preprocessed.to_csv(path_file_export + 'out_dblp_papers_and_locations_final.csv')
print(f'Successfully Exported the Preprocessed CSV to {path_file_export}out_dblp_papers_and_locations_final.csv')

Check of the Exported CSV to be sure that everything went fine.

In [ ]:
# Check of the Exported CSV
df_dblp_exported_csv = pd.read_csv(path_file_export + 'out_dblp_papers_and_locations_final.csv', low_memory=False, index_col=[0])
df_dblp_exported_csv